In [ ]:
!pip install transformers torch pyspark tensorflow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 2.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using ca

In [ ]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


# Giai đoạn 1

In [ ]:
!pip install --upgrade tensorflow
!pip install --upgrade keras

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 590.6/590.6 MB 989.6 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 76.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 73.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 89.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 65.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.2/311.2 kB 38.1 MB/s eta 0:00:00
  Attempting uninstall: ml-dtypes
    Found existing installation: ml-dtypes 0.2.0
    Uninstalling ml-dtypes-0.2.0:
      Successfully uninstalled ml-dtypes-0.2.0
  Attempting uninstall: h5py
    Found existing installation: h5py 3.9.0
    Uninstalling h5py-3.9.0:
      Successfully uninstalled h5py-3.9.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.15.2
    Uninstalling tensorboard-2.15.2:
      Successfully uninstalled tensorboard-2.15.2
  Attempting uninstall: keras
    

^C


In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf
from pyspark.sql.types import IntegerType
import logging
# Khởi tạo Spark Session
spark = SparkSession.builder \
    .appName("Stage1_Inference") \
    .getOrCreate()

# Định nghĩa các hàm inference của bạn
def get_prediction_stage1(sentence):
    import numpy as np
    import tensorflow as tf
    #from tensorflow.keras.preprocessing.text import Tokenizer
    from tensorflow.keras.preprocessing.sequence import pad_sequences
    import joblib
    import pickle


    logging.info(f"Processing sentence: {sentence}")

    # Load the model
    model = tf.keras.models.load_model("/content/drive/MyDrive/project/model/LSTM_rest.h5")

    # Load the tokenizer
    #tokenizer = joblib.load('/content/drive/MyDrive/Study/Năm 3/DS200 - Big data/project/model/tokenizer.pkl')
    with open('/content/drive/MyDrive/project/model/tokenizer.pkl', 'rb') as f:
      tokenizer = pickle.load(f)

    # Tokenize and pad the input sentence
    max_sequence_length = 100
    sentence_tokenized = tokenizer.texts_to_sequences([sentence])
    logging.info(f"Tokenized sentence: {sentence_tokenized}")
    sentence_padded = pad_sequences(sentence_tokenized, maxlen=max_sequence_length, padding='post')
    logging.info(f"Padded sentence: {sentence_padded}")

    # Predict the label
    prediction = model.predict(sentence_padded)
    logging.info(f"Prediction: {prediction}")

    # Convert the predicted probabilities to class labels
    predicted_label = 1 if prediction[0][1] > 0.5 else 0

    return int(predicted_label)

# Đăng ký hàm inference như một UDF
stage1_udf = udf(get_prediction_stage1, IntegerType())

# Tạo DataFrame mẫu
data = [("I waited for 10-15 minutes for service ordered a beer & was never served again .",)]
columns = ["review"]
df = spark.createDataFrame(data, columns)

# Áp dụng UDF lên DataFrame
df_with_predictions = df.withColumn("prediction", stage1_udf(df.review))

# Hiển thị kết quả
df_with_predictions.show(truncate=False)

+--------------------------------------------------------------------------------+----------+
|review                                                                          |prediction|
+--------------------------------------------------------------------------------+----------+
|I waited for 10-15 minutes for service ordered a beer & was never served again .|1         |
+--------------------------------------------------------------------------------+----------+



# Giai đoạn 2

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType

# Khởi tạo Spark Session
spark = SparkSession.builder \
    .appName("Stage2_Inference") \
    .getOrCreate()

# Định nghĩa các hàm inference của bạn
def get_prediction_stage2(review):
    from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
    import torch

    model_id = 'ThuyNT03/DS200-big_train-res16-T5'
    tokenizer_T5 = AutoTokenizer.from_pretrained(model_id)
    model_T5 = AutoModelForSeq2SeqLM.from_pretrained(model_id)

    max_input_length = 256
    max_output_length = 256

    input_ids = tokenizer_T5(review, max_length=max_input_length, return_tensors="pt", padding="max_length", truncation=True).input_ids
    outputs = model_T5.generate(input_ids=input_ids, max_new_tokens=max_output_length, do_sample=True)
    pred = tokenizer_T5.batch_decode(outputs.detach().cpu().numpy(), skip_special_tokens=True)[0]

    return pred

# Đăng ký hàm inference như một UDF
stage2_udf = udf(get_prediction_stage2, StringType())

# Tạo DataFrame mẫu
data = [("I waited for 10-15 minutes for service ordered a beer & was never served again .",)]
columns = ["review"]
df = spark.createDataFrame(data, columns)

# Áp dụng UDF lên DataFrame
df_with_predictions = df.withColumn("prediction", stage2_udf(df.review))

# Hiển thị kết quả
df_with_predictions.show(truncate=False)


+--------------------------------------------------------------------------------+---------------------------------------------------------+
|review                                                                          |prediction                                               |
+--------------------------------------------------------------------------------+---------------------------------------------------------+
|I waited for 10-15 minutes for service ordered a beer & was never served again .|NULL, service general, negative, waited for 10-15 minutes|
+--------------------------------------------------------------------------------+---------------------------------------------------------+

